In [26]:
#ML Tlachac, Left on Read, 2022
#https://github.com/mltlachac/UbiComp2022

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from scipy import stats
import collections
import operator
import argparse
import random
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn import metrics
from statistics import mean 
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn import utils
from sklearn.datasets import load_digits
from sklearn import svm
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.decomposition import PCA, KernelPCA, NMF

In [96]:
name = "latencyCAT"
data = pd.read_csv("cleanedLatencyTexts14CAT.csv")
low = 7
split = 10
label = "phq" #gad

dfindex = 2
if name == "latencyCAT":
    dfindex = 3
    data = data.drop(columns=['mean'])

print(data.shape)

#binary labels
d10 = []
for i in range(0, data.shape[0]):
    if int(data[label][i]) >= split:
        d10.append(1)
    else:
        d10.append(0)
data[label] = d10

data = data[data.responses >= low].reset_index(drop=True)
print(data.shape)

data.head()

(69, 12)
(49, 12)


,id,phq,gad,contacts,responses,min,quant10,quant25,quant50,quant75,quant90,max
0,7KBUI5ZGZ,1,14,21,41,6,10.0,28.00,222.0,1676.0,3108.0,74839
1,Q6N7PT68V,1,8,14,76,0,12.0,32.50,87.0,2144.0,20518.5,77510
2,SKNKBM56D,1,10,35,13,9,14.4,21.00,30.0,63.0,109.6,1070
3,33WM31B9D,0,4,30,120,8,22.0,46.75,115.5,507.5,3038.1,45840
4,EHKQK9NGT,0,7,10,12,7,25.3,52.00,78.0,170.5,227.2,459


In [80]:
featureEs = "PCA"
modelTypelist = ["NB","LR1","SVC1", "kNN5", "RF", "XG"]
    
#create lists to populate
flist = [] 
mlist = []
llist = []
featureList = []
f1List = []
accuracyList = []
truePosList = []
trueNegList = []
falsePosList = []
falseNegList = []
predictions = []
rseed = []

for r in range(0,data.shape[0]):
    testdata = data[r:r+1]
    traindata = data.drop([r])

    #limit to features
    testContent = testdata[testdata.columns[dfindex:]]
    print(testContent.shape)
    trainContent = traindata[traindata.columns[dfindex:]]
    print(trainContent.shape)
    
    #NEED TO SCALE BEFORE FEATURE SELECTION/REDUCATION
    min_max_scaler = preprocessing.MinMaxScaler()  
    np_scaled = min_max_scaler.fit_transform(trainContent)
    featureSubset = pd.DataFrame(np_scaled)
    np_scaled2 =  min_max_scaler.transform(testContent)
    testSubset = pd.DataFrame(np_scaled2)

    target = list(traindata[label])
    y_test = list(testdata[label])


    featureDF = []
    testDFs = []

    if featureE == "PCA":
        nFeatureList = list(np.arange(1,9,1))
        for numberOfFeatures in nFeatureList:
            pca = PCA(n_components=numberOfFeatures)
            pca = pca.fit(featureSubset)
            X_pca = pca.transform(featureSubset)
            pcaDF = pd.DataFrame(X_pca)
            pcaDF = pcaDF.assign(target = target)
            featureDF.append(pcaDF)
            testSubset2 = pca.transform(testSubset)
            testDFs.append(pd.DataFrame(testSubset2))

    for f in range(0, len(featureDF)):

        train_phq9 = featureDF[f]
        X_test = testDFs[f]

        # upsampling 
        #Count 1s and 0s
        ones = len(train_phq9.loc[train_phq9['target'] == 1])
        zeros = len(train_phq9.loc[train_phq9['target'] == 0])
        if ones >= zeros:
            majority = 1
            minority = 0
        else:
            majority = 0
            minority = 1


        # Upsample TrainingSet 
        train_majority = train_phq9[train_phq9.target==majority]
        train_minority = train_phq9[train_phq9.target==minority]

        #print("train_majority ="  + str(len(train_majority)))
        #print("train_minority ="  + str(len(train_minority)))

        # Upsample minority class
        train_minority_upsampled = resample(train_minority, 
                                         replace=True,     # sample with replacement
                                         n_samples=len(train_majority),    # to match majority class
                                         random_state=r) # reproducible results

        # Combine majority class with upsampled minority class
        train_phq9 = pd.concat([train_majority, train_minority_upsampled])

        #seperate features and target
        y_train = train_phq9["target"]
        X_train = train_phq9.drop(columns = "target")

        for modelType in modelTypelist:

            #add data to lists
            llist.append(label)
            featureList.append(f +1)
            mlist.append(modelType)

            #chose model type
            if modelType == "SVC1":
                clf = svm.SVC(kernel='rbf', random_state=r)
            elif modelType == "SVC2":
                clf = svm.SVC(kernel='linear', random_state=r)
            elif modelType == "SVC3":
                clf = svm.SVC(kernel='poly', random_state=r)
            elif modelType == "SVC4":
                clf = svm.SVC(kernel='sigmoid', random_state=r)
            elif modelType == "kNN3":
                clf = KNeighborsClassifier(n_neighbors=3)
            elif modelType == "kNN5":
                clf = KNeighborsClassifier(n_neighbors=5)
            elif modelType == "kNN1":
                clf = KNeighborsClassifier(n_neighbors=1)
            elif modelType == "LR1":
                clf = LogisticRegression(random_state=r)
            elif modelType == "NB":
                clf = GaussianNB()
            elif modelType == "RF":
                clf = RandomForestClassifier(random_state=r)
            elif modelType == "XG":
                clf = xgb.XGBClassifier(random_state=r)
            else:
                print("YOU MADE A MISTAKE!!!")


            #train model and make predictions
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)

            TP = 0
            TN = 0
            FP = 0
            FN = 0
            if (y_pred[0]==1) & (y_test[0]==1):
                TP = 1
            elif (y_pred[0]==0) & (y_test[0]==0):
                TN = 1
            elif (y_pred[0]==1) & (y_test[0]==0):
                FP = 1
            elif (y_pred[0]==0) & (y_test[0]==1):
                FN = 1
            else:
                print("ERROR")

            #populate lists with results
            truePosList.append(TP)
            trueNegList.append(TN)
            falsePosList.append(FP)
            falseNegList.append(FN)
            rseed.append(r)

resultsDF = pd.DataFrame()
resultsDF["label"] = llist
resultsDF["model"] = mlist
resultsDF["nFeatures"] = featureList
resultsDF["truePos"] = truePosList
resultsDF["trueNeg"] = trueNegList
resultsDF["falsePos"] = falsePosList
resultsDF["falseNeg"] = falseNegList
resultsDF["testIndex"] = rseed

resultsDF.to_csv("results2/" + name + label + str(split) + "low" + str(low) + ".csv")

(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
(1, 9)
(48, 9)
